<a href="https://colab.research.google.com/github/nsanidhya1/python/blob/main/Copy_of_Untitled12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install --upgrade transformers torch gradio sentencepiece


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 MB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.6/324.6 kB 23.8 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import pipeline
device = 0 if torch.cuda.is_available() else -1

summarizer = pipeline("summarization", model="facebook/bart-large-cnn", device=device)

qa_pipeline = pipeline("question-answering", model="distilbert-base-cased-distilled-squad", device=device)

print("CUDA available:", torch.cuda.is_available())
print("Summarizer and QA pipelines are ready.")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

Device set to use cpu


CUDA available: False
Summarizer and QA pipelines are ready.


In [ ]:
import re

def split_into_chunks(text, max_words=300):
    # Simple, reliable chunking by words for long articles
    words = text.split()
    for i in range(0, len(words), max_words):
        yield " ".join(words[i:i+max_words])

def summarize_long_text(text, max_length=120, min_length=30, do_sample=False):
    text = re.sub(r"\s+", " ", text).strip()
    if not text:
        return ""
    chunks = list(split_into_chunks(text, max_words=300))
    parts = []
    for chunk in chunks:
        out = summarizer(chunk, max_length=max_length, min_length=min_length, do_sample=do_sample)
        parts.append(out[0]["summary_text"])
    # final pass to summarize the concatenated partial summaries
    combined = " ".join(parts)
    if len(parts) > 1 and len(combined.split()) > 250:
        final = summarizer(combined, max_length=max_length, min_length=min_length, do_sample=do_sample)
        return final[0]["summary_text"]
    return combined


In [ ]:
import gradio as gr

def do_summary(text, max_len, min_len, do_sample):
    if not text or not text.strip():
        return "Please paste some text to summarize."
    try:
        return summarize_long_text(text, max_length=max_len, min_length=min_len, do_sample=do_sample)
    except Exception as e:
        return f"Error during summarization: {e}"

def do_qa(context, question):
    if not context or not context.strip():
        return "Please paste some context (a paragraph or article).", 0.0
    if not question or not question.strip():
        return "Please type a question.", 0.0
    try:
        out = qa_pipeline(question=question, context=context)
        return out["answer"], float(out["score"])
    except Exception as e:
        return f"Error during Q&A: {e}", 0.0

with gr.Blocks(title="GenAI: Summarizer + Q&A") as demo:
    gr.Markdown("# 🧠 Generative AI (Summarizer + Q&A)\nBuilt with Python, Transformers, and Gradio")

    with gr.Tab("Summarization"):
        gr.Markdown("Paste long text; we’ll auto-chunk if needed.")
        summary_input = gr.Textbox(lines=12, label="Input Text")
        with gr.Row():
            max_len = gr.Slider(32, 256, value=120, step=1, label="Max summary length (tokens-ish)")
            min_len = gr.Slider(8, 128, value=30, step=1, label="Min summary length (tokens-ish)")
        do_sample = gr.Checkbox(value=False, label="Enable sampling (more creative)")
        summary_btn = gr.Button("Summarize")
        summary_output = gr.Textbox(lines=8, label="Summary")

        summary_btn.click(do_summary, inputs=[summary_input, max_len, min_len, do_sample], outputs=[summary_output])

    with gr.Tab("Q&A"):
        gr.Markdown("Paste context and ask a question.")
        qa_context = gr.Textbox(lines=12, label="Context")
        qa_question = gr.Textbox(lines=2, label="Question")
        qa_btn = gr.Button("Answer")
        qa_answer = gr.Textbox(lines=2, label="Answer")
        qa_conf = gr.Number(label="Confidence (model score)")

        qa_btn.click(do_qa, inputs=[qa_context, qa_question], outputs=[qa_answer, qa_conf])

demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://c3f4785f3fb17f258b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
import gradio as gr

def read_txt(file_obj):
    if file_obj is None:
        return ""
    return file_obj.decode("utf-8", errors="ignore")

def summarize_file(file, max_len, min_len, do_sample):
    try:
        text = read_txt(file)
        if not text.strip():
            return "Uploaded file is empty."
        return summarize_long_text(text, max_length=max_len, min_length=min_len, do_sample=do_sample)
    except Exception as e:
        return f"Error: {e}"

def qa_from_file(file, question):
    try:
        context = read_txt(file)
        if not context.strip():
            return "Uploaded file is empty.", 0.0
        if not question.strip():
            return "Please type a question.", 0.0
        out = qa_pipeline(question=question, context=context)
        return out["answer"], float(out["score"])
    except Exception as e:
        return f"Error: {e}", 0.0

with gr.Blocks(title="GenAI: Summarizer + Q&A (with file upload)") as demo2:
    gr.Markdown("# 📄 Summarize or Q&A from a .txt file")

    with gr.Tab("Summarize .txt"):
        file_up = gr.File(label="Upload .txt", file_types=[".txt"])
        with gr.Row():
            max_len2 = gr.Slider(32, 256, value=120, step=1, label="Max summary length")
            min_len2 = gr.Slider(8, 128, value=30, step=1, label="Min summary length")
        do_sample2 = gr.Checkbox(value=False, label="Enable sampling")
        btn_sum = gr.Button("Summarize File")
        out_sum = gr.Textbox(lines=8, label="Summary")
        btn_sum.click(summarize_file, inputs=[file_up, max_len2, min_len2, do_sample2], outputs=[out_sum])

    with gr.Tab("Q&A on .txt"):
        file_up2 = gr.File(label="Upload .txt", file_types=[".txt"])
        ques2 = gr.Textbox(lines=2, label="Question")
        btn_qa2 = gr.Button("Answer")
        out_ans2 = gr.Textbox(lines=2, label="Answer")
        out_score2 = gr.Number(label="Confidence (score)")
        btn_qa2.click(qa_from_file, inputs=[file_up2, ques2], outputs=[out_ans2, out_score2])

demo2.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://29616566567107644f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
